In [25]:
import pandas as pd
import numpy as np
from datetime import date
import datetime
import itertools
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

import warnings
import locale

locale.setlocale(locale.LC_TIME, 'es_MX.UTF-8')
warnings.filterwarnings("ignore")

In [26]:
def ObtenerArchivo(diario = True):
                                        #vars
    mes = datetime.datetime.now().strftime("%B") #Obteniendo el mes actual
    archivo_ventas = f'ventas_diarias_{mes}.xlsm' #Nombre del archivo
    archivo_compras = f'compras_diarias_{mes}.xlsm' #Nombre del archivo
    #Ruta de los archivos 
    ruta_archivo_ventas = f'{mes}/{archivo_ventas}' #[A modificar]
    ruta_archivo_compras = f'{mes}/{archivo_compras}' #[A modificar]
    
    #Archivos Excel
    excel_ventas = pd.ExcelFile(ruta_archivo_ventas, engine = 'openpyxl')
    excel_compras = pd.ExcelFile(ruta_archivo_compras, engine = 'openpyxl')
    
    if diario:
        hoja = str(date.today().strftime("%d_%m_%Y")) #Hoja del archivo excel del dia de hoy
        return excel_ventas, excel_compras, hoja
    else:
        hojas_venta = [hoja for hoja in excel_ventas.sheet_names if '_20' in hoja]#Obteniendo los nombres de las hojas del archivo Excel
        hojas_compra = [hoja for hoja in excel_ventas.sheet_names if '_20' in hoja]
        return excel_ventas, excel_compras, hojas_venta, hojas_compra

In [27]:
def LimpiarDatos(datos: pd.DataFrame):
    #Cambiando formato fecha
    datos['fecha'] = datos['fecha'].dt.strftime('%d/%m/%Y')
    
    #Quitando columnas extras
    for column in datos.columns:
        if column not in ['fecha','cliente','cantidad','producto','precio_u','importe']:
            datos.drop(column,
                       axis = 1,
                       inplace = True)
            
    #Quitando filas vacias
    datos.dropna(how='all', 
                 inplace = True)
    return datos

In [28]:
def Graficar(datos: pd.DataFrame, titulo: str, valores):
    fig = px.pie(datos, 
                 values=valores, 
                 names=datos.index, 
                 title= titulo)
    fig.update_traces(textinfo='value')
    fig.update_layout(title_font_size=30)
    fig.show()

In [36]:
def Reporte(ventas, compras: pd.DataFrame, diario = True):
    #Total $ en ventas
    total_vendido = ventas['importe'].sum()
    print(f"TOTAL VENDIDO: ${total_vendido:,.2f}")
    #Total $ en compras
    total_comprado = compras['importe'].sum()
    print(f"TOTAL COMPRADO: ${total_comprado:,.2f}")
    #Cuanto compro cada cliente de cada producto
    ventas_por_cliente = ventas.groupby(['cliente'])['producto','importe'].sum().sort_values('importe', ascending=False)
    display(ventas_por_cliente) 
    #Cuanto se vendio de cada cosa
    ventas_por_producto = ventas.groupby('producto')['cantidad','importe'].sum().sort_values('importe', ascending=False)
    display(ventas_por_producto) 
    #Que se compro y cuanto se compro
    compras_por_producto = compras.groupby('producto')['cantidad', 'importe'].sum().sort_values('importe', ascending=False)
    display(compras_por_producto) 
    
    #Graficando
    if diario:
        fecha = str(date.today().strftime("%d_%m_%Y"))
        #Graficas Ventas
        Graficar(ventas_por_cliente, 
                 f'{fecha}: Ventas por Cliente [$]',
                 'importe')
        Graficar(ventas_por_producto,
                 f'{fecha}: Ventas por producto [$]',
                 'importe')
        
        Graficar(ventas_por_producto, 
                 f'{fecha}: Ventas por producto [KG]',
                 'cantidad')   
        
        #Graficas Compras
        Graficar(compras_por_producto, 
                 f'{fecha}: Compras [$]',
                 'importe') 
        Graficar(compras_por_producto, 
                 f'{fecha}: Compras [KG]',
                 'cantidad')   
        
    else:
        fecha = str(date.today().strftime("%B")).capitalize()
        Graficar(ventas_por_cliente, 
                 f'{fecha}: Ventas por Cliente [$]',
                 'importe')
        Graficar(ventas_por_producto,
                 f'{fecha}: Ventas por producto [$]',
                 'importe')
        
        Graficar(ventas_por_producto, 
                 f'{fecha}: Ventas por producto [KG]',
                 'cantidad')   
        
        #Graficas Compras
        Graficar(compras_por_producto, 
                 f'{fecha}: Compras [$]',
                 'importe') 
        Graficar(compras_por_producto, 
                 f'{fecha}: Compras [KG]',
                 'cantidad')  

--- 

In [37]:
def Reporte_de_Hoy():  
    #Obteniendo archivo de ventas
    excel_ventas,excel_compras, hoja = ObtenerArchivo()
    #Obteniendo datos de hoy
    ventas_hoy = LimpiarDatos(excel_ventas.parse(hoja))
    compras_hoy = LimpiarDatos(excel_compras.parse(hoja))
    inventario = {'pechuga': 1000,
                 'ala natural': 1000,
                 'pierna con muslo': 1000,
                 'pechuga caja': 1000,
                 'pollo entero': 1000}
    
    Reporte(ventas_hoy, compras_hoy)
Reporte_de_Hoy()

TOTAL VENDIDO: $0.00
TOTAL COMPRADO: $51,007.22


,producto,importe
cliente,,


,cantidad,importe
producto,,


,cantidad,importe
producto,,
pollo entero,652.82,33293.82
pechuga caja,203.80,9493.20
pulpa,124.60,8099.00
tronco,10.10,121.20


In [38]:
def Reporte_del_Mes():
    excel_ventas, excel_compras, hojas_venta, hojas_compra = ObtenerArchivo(diario = False)
    
    #Obteniendo  y Agrupando datos
    ventas_del_mes = pd.DataFrame()
    compras_del_mes = pd.DataFrame()
    
    for hoja in hojas_venta:
        venta = LimpiarDatos(excel_ventas.parse(hoja))
        ventas_del_mes = ventas_del_mes.append(venta)
        
    for hoja in hojas_compra:
        compra = LimpiarDatos(excel_compras.parse(hoja))
        compras_del_mes = compras_del_mes.append(compra)   
        
    Reporte(ventas_del_mes,compras_del_mes, diario = False)
    
Reporte_del_Mes()

TOTAL VENDIDO: $0.00
TOTAL COMPRADO: $51,007.22


,producto,importe
cliente,,


,cantidad,importe
producto,,


,cantidad,importe
producto,,
pollo entero,652.82,33293.82
pechuga caja,203.80,9493.20
pulpa,124.60,8099.00
tronco,10.10,121.20
